In [ ]:
#L ← Empty list that will contain the sorted elements
#S ← Set of all nodes with no incoming edge

#while S is not empty do
#    remove a node n from S
#    add n to L
#    for each node m with an edge e from n to m do
#        remove edge e from the graph
#        if m has no other incoming edges then
#            insert m into S

#if graph has edges then
#    return error   (graph has at least one cycle)
#else 
#    return L   (a topologically sorted order)

import numpy as np

seed = 123
np.random.seed(seed)

def topsort(G):
    L = []
    S = []
    edges = list(G.edges())
    print(edges)
    nodes = list(G.nodes())
    predecessors = dict([])
    successors = dict([])
    for n in nodes:
        predecessors[n] = list(G.predecessors(n))
        successors[n] = list(G.successors(n))
        if len(predecessors[n])==0:
            S.append(n)
    #print(S)        
    while len(S)>0:
        #print(S,L)
        n = S.pop(0)
        L.append(n)
        succ = [x for x in successors[n]]
        
        for m in successors[n]:
            pred = [x  for x in predecessors[m] if x!=n]
            edges.remove((n,m))
            #succ.remove(m)
            
            if pred==[]:
                #print(m)
                succ.remove(m)
                S.append(m)
            predecessors[m] = [x for x in pred]    
        #print(succ)
        successors[n] = [s for s in succ]
    if len(edges)>0:
        #print(edges)
        print(len(L),L)
        return None
    return L

def getRandomAcyclicPath(G,maxlen=100,seed=123):
    
    
    np.random.seed(seed)
    path = []
    v = np.random.choice(G.nodes())
    while not ((v in path) and len(path)<=maxlen):
        if v in path:
            break
        path.append(v)
        scc = list(G.successors(v))
    #print(v,">",G[v],",",path)
        if len(scc)>0:
            v = np.random.choice(scc)
    return path


def deming_regression(xx,yy):
    import numpy as np
    
    n = len(xx)
    xhat = 1.0/n * sum(xx)
    yhat = 1.0/n * sum(yy)
    s2x = 1.0/(n-1)*sum([(xx[i]-xhat)**2 for i in range(len(xx))])
    s2y = 1.0/(n-1)*sum([(yy[i]-yhat)**2 for i in range(len(yy))])
    sxy = 1.0/(n-1)*sum([(yy[i]-yhat)*(xx[i]-xhat) for i in range(len(yy))])
    
    if abs(sxy)<0.01:
        sxy = 0.01
    if s2y**2-2*s2y*s2x+s2x**2+4*1*sxy**2 <0.0:
        sr = 0.0
    else:
        sr = np.sqrt(s2y**2-2*s2y*s2x+s2x**2+4*1*sxy**2)    
    
    beta1 = (s2y-1*s2x+sr)/(2*sxy)
    
    beta0 = yhat-beta1*xhat
    
    xstar = [xx[i]+beta1/(beta1**2+1)*(yy[i]-beta0-beta1*xx[i]) for i in range(len(xx))]
    return (beta1,beta0,xstar)


def pathCost(path,cost,verbose=False):
    s = 0
    for i in range(len(path)-1):
        u,v = path[i],path[i+1]
        s+=cost[(u,v)]
        if verbose: print("pathcost, u,v,cost = ",u,v,cost[(u,v)])
    return s    


def is_prime(n):
    for k in range(2,1+int(np.floor(np.sqrt(n)))):
        if n%k ==0 and 1 < k < n:
            return False
    return n!=1


def primes(n):
    return [x for x in range(1,n+1) if is_prime(x)]

def prime_divisors(n):
    pd = []
    for k in range(1,1+int(np.floor(np.sqrt(n)))):
        if n%k==0:
            if is_prime(k):
                pd.append(k)
            if is_prime(n//k) and k!=n//k:
                pd.append(n//k)
    return pd

def gcd(a,b):
    return int(np.gcd(a,b))

def valuation(n,p):
    v = 0
    while n%p==0:
        v += 1
        n = n//p
    return v    


def kk(a,b):
    return sum([valuation(a,p)*valuation(b,p) for p in prime_divisors(gcd(a,b))])

def dist(a,b):
    return np.sqrt(kk(a,a)+kk(b,b)-2*kk(a,b))


print([prime_divisors(n) for n in range(1,11)])

def GG(n,f=1,rnd=True):
    import numpy as np
    G = nx.DiGraph()
    for a in range(1,n+1):
        G.add_node(a)
    for a in range(1,n+1):        
        for p in primes(n//a):
            b = a*p
            G.add_edge(a,b,weight=np.round(dist(a,b),3))
            #if a%b==0 and is_prime(a/b):
            #    G.add_edge(a,b,abs(a-b))
    return G

def projectGraph(G,S,paths,verbose=False):
    #xx = [pos[a][0] for a in G.nodes()]
    #yy = [pos[a][1] for a in G.nodes()]
    
    rvec = 2**20 # Richtungsvektor der Geraden durch den Ursprung
    
    #x0,y0 = np.array([x0,y0])-img.shape[0]*rvec
    #x1,y1 = np.array([x1,y1])+img.shape[1]*rvec
    #rvec = np.array([x1-x0,y1-y0])
    newpos = dict([])
    gN = list(G.nodes())
    mapG = dict(zip(range(len(gN)), gN))
    for i in range(len(G.nodes())):
        # projektion auf Gerade durch Ursprung mit Richtungsvektor rvec
        newpos[mapG[i]] = kk(rvec,mapG[i])/kk(rvec,rvec) 
        

    cost = dict([])    
    tops = (list(nx.topological_sort(G)))
    
    xsorted = sorted([(kk(rvec,mapG[i]),mapG[i]) for i in range(len(G.nodes()))])
    isorted = [x[1] for x in xsorted]
    if verbose: print("isorted =", isorted)

    zipped = dict(list(zip(tops,isorted)))

    if verbose:
        print("topsort = ",tops)
        print("xsorted = ",xsorted)
        print("zipped = ", zipped)

            
        
    newcost = dict([])
    #cost = dict([])

    myG = nx.DiGraph()

    #dm = nx.diameter(G)
    
    minCost = np.inf
    
    for e in G.edges():
        u,v = e
        myG.add_node(u)
        myG.add_node(v)
        if not (u,v) in cost.keys():
            cost[(u,v)] = dist(u,v)
        if minCost>cost[(u,v)]:
            minCost = cost[(u,v)]
        if not (u in S.nodes() and v in S.nodes()): #if not (u,v) in S.edges():
            if verbose: print(" not in S (u,v) = ",u,v)
            newcost[(u,v)] = cost[(u,v)]
            myG.add_edge(u,v,weight=newcost[(u,v)])
        if u in S.nodes() and v in S.nodes():#if (u,v) in S.edges():
            if verbose: print("u,v = ",u,v)
            U,V = zipped[u],zipped[v]
            newcost[(u,v)] = abs(newpos[U]-newpos[V]) 
            myG.add_edge(u,v,weight=newcost[(u,v)])
            
    #return myG,newcost,newpos        

    # determine constant for paths    
    minPathDist = np.inf
    maxPathDist = -np.inf
    reweightFactor = np.inf
    eps = 0.001
    #for path in paths:
    #    u,v= path[0],path[-1]
    #    dist = nx.shortest_path_length(myG,u,v,weight="weight")
    #    sdist = pathCost(path,newcost)
    #    if abs(sdist-dist)>eps:
    #        if reweightFactor > dist/sdist:
    #            reweightFactor = dist/sdist
    inds = [v for v in S.nodes()]
    radius = abs(newpos[xsorted[0][1]]-newpos[xsorted[-1][1]])
    reweightFactor = minCost / radius
    
    if verbose:
        print("reweightFactor = ",reweightFactor)
        print("radius = ",radius)
        print("minCost = ",minCost)
            
    if True:
        for e in myG.edges():        
            u,v = e
            if u in S.nodes() and v in S.nodes():#
                if verbose :
                    print("reweighting: (u,v) = ",(u,v),newcost[(u,v)],newcost[(u,v)]*reweightFactor)
                nx.set_edge_attributes(myG,{(u,v):{"weight":newcost[(u,v)]*reweightFactor}})
                newcost[(u,v)]= newcost[(u,v)]*reweightFactor
        
    return myG,newcost,newpos

def smallGraph():
    G = nx.DiGraph()
    
    G.add_nodes_from(list(range(9+1)))
    
    G.add_edges_from(
        [ (0,1),
          (1,2),
          (2,3),
          (3,4),(4,5),
          (0,6),(6,5),
          (0,7),(7,4),
          (0,8),(8,3),
          (0,9),(9,2)
        ]
    )
    pos = {
        0: (0,0), 1 : (1,0),2:(1,-1),3:(-1,-1),4:(-1,1),5:(1,1),
        6:(1/2,1/2),7:(1/2,-1/2),8:(-1/2,-1/2),9:(-1/2,1/2)
    }
    pos = dict([(k,np.array(pos[k])) for k in pos.keys()])
    return G, pos

def checkShortestPathProperty(myG,S,cost,randomShortestPaths,paths,verbose=False):
    eps = 1e-4
    nrErrors = 0
    if verbose: 
        print("paths = ", paths)
    for path in randomShortestPaths:
        u,v= path[0],path[-1]
        dist = nx.shortest_path_length(myG,u,v,weight="weight")
        sdist = pathCost(path,cost)
        if abs(sdist-dist)>eps and path in paths:#and all([path[i] in S.vertices() for i in range(len(path))]):
            print("Error:",path, sdist,dist)
            nrErrors += 1
        elif abs(sdist-dist)>eps and not path in paths: #all([path[i] in S.vertices() for i in range(len(path))]):
            if verbose: print("not a shortest path as required:",path,sdist,dist)
        elif abs(sdist-dist)<=eps  and path in paths: #all([path[i] in S.vertices() for i in range(len(path))]):
            if verbose: print("shortest path as required:",path,sdist,dist)
        elif abs(sdist-dist)<=eps  and not path in paths: #all([path[i] in S.vertices() for i in range(len(path))]):
            if verbose: print("shortest path but not required:",path,sdist,dist)
    print("nrErrors = ",nrErrors)        
    return nrErrors
    

In [ ]:
# topologische Sortierung
import time
import networkx as nx
from copy import deepcopy 

datapoints = []
number_of_nodes  = []
graph_creation_times = []
graph_projection_times = []
random_path_times = []

nrnodes = [100,500,1000,5000,10**4,10**5,10**6]
nrnodes = [1000,2000,10**4]
verbose=True
printIt = True
for nrn in nrnodes:
    print("number of nodes = ",nrn)
    start = time.time()
    G = GG(nrn,f=10) #smallGraph() #GG(nrn)
    end = time.time()
    print("Graph created in time = ",end-start)
    Gs = deepcopy(G)
    #ox.plot_graph(G)
    #paths = [ getRandomAcyclicPath(G) for k in range(140)]
    H = Gs
    number_of_nodes.append(nrn)
    graph_creation_times.append(end-start)


    nrrndpaths = [0.2]
    for nrrnd in nrrndpaths:
        start = time.time()
        randomShortestPaths = [getRandomAcyclicPath(H,maxlen=100,seed=k+1) for k in range(int(np.floor(nrrnd*nrn)))]
        paths = randomShortestPaths[0:len(randomShortestPaths)//2]
        end = time.time()
        print("random paths chosen in time = ",end-start)
        random_path_times.append(end-start)
        datapoints.append((nrn,len(randomShortestPaths),len(paths)))
        edges = []
        verts = []
        for p in paths:
            if len(p)==1:
                continue
            for i in range(len(p)-1):
                verts.append(p[i])
                edges.append((p[i],p[i+1]))
            if len(p)>1: 
                verts.append(p[i+1])    
        S = nx.DiGraph()
        S.add_nodes_from(verts)
        S.add_edges_from(edges)

        #wcc = list(nx.weakly_connected_components(T))
        #print(wcc)
        #if len(wcc)==0:
        #    S = DiGraph(T)
        #else:    
        #    comp = wcc[0]
        #    S= DiGraph(nx.subgraph(T,comp))
        
  

        print("projecting graph")
        start = time.time()
        myG,newcost,newpos = projectGraph(G,S,paths,verbose=verbose)
        end = time.time()
        graph_projection_times.append(end-start)

        print("graph projected in time = ",end-start)

        checkShortestPath = printIt
        if checkShortestPath:
            checkShortestPathProperty(myG,S,newcost,randomShortestPaths,paths,verbose=verbose)

        
print(datapoints)            
print(graph_creation_times)
print(random_path_times)
print(graph_projection_times)

 not in S (u,v) =  109 327
 not in S (u,v) =  109 545
 not in S (u,v) =  109 763
 not in S (u,v) =  109 1199
 not in S (u,v) =  109 1417
 not in S (u,v) =  109 1853
 not in S (u,v) =  110 220
 not in S (u,v) =  110 330
 not in S (u,v) =  110 550
 not in S (u,v) =  110 770
 not in S (u,v) =  110 1210
 not in S (u,v) =  110 1430
 not in S (u,v) =  110 1870
 not in S (u,v) =  111 222
 not in S (u,v) =  111 333
 not in S (u,v) =  111 555
 not in S (u,v) =  111 777
 not in S (u,v) =  111 1221
 not in S (u,v) =  111 1443
 not in S (u,v) =  111 1887
 not in S (u,v) =  112 224
 not in S (u,v) =  112 336
 not in S (u,v) =  112 560
 not in S (u,v) =  112 784
 not in S (u,v) =  112 1232
 not in S (u,v) =  112 1456
 not in S (u,v) =  112 1904
 not in S (u,v) =  113 226
 not in S (u,v) =  113 339
 not in S (u,v) =  113 565
 not in S (u,v) =  113 791
 not in S (u,v) =  113 1243
 not in S (u,v) =  113 1469
 not in S (u,v) =  113 1921
 not in S (u,v) =  114 228
 not in S (u,v) =  114 342
 not in S (u,

 not in S (u,v) =  320 1600
 not in S (u,v) =  321 642
 not in S (u,v) =  321 963
 not in S (u,v) =  321 1605
 not in S (u,v) =  322 644
 not in S (u,v) =  322 966
 not in S (u,v) =  322 1610
 not in S (u,v) =  323 646
 not in S (u,v) =  323 969
 not in S (u,v) =  323 1615
 not in S (u,v) =  324 648
 not in S (u,v) =  324 972
 not in S (u,v) =  324 1620
 not in S (u,v) =  325 650
 not in S (u,v) =  325 975
 not in S (u,v) =  325 1625
 not in S (u,v) =  326 652
 not in S (u,v) =  326 978
 not in S (u,v) =  326 1630
 not in S (u,v) =  327 654
 not in S (u,v) =  327 981
 not in S (u,v) =  327 1635
 not in S (u,v) =  328 656
 not in S (u,v) =  328 984
 not in S (u,v) =  328 1640
 not in S (u,v) =  329 658
 not in S (u,v) =  329 987
 not in S (u,v) =  329 1645
 not in S (u,v) =  330 660
 not in S (u,v) =  330 990
 not in S (u,v) =  330 1650
 not in S (u,v) =  331 662
 not in S (u,v) =  331 993
 not in S (u,v) =  331 1655
 not in S (u,v) =  332 664
 not in S (u,v) =  332 996
 not in S (u,v) 

 not in S (u,v) =  875 1750
 not in S (u,v) =  876 1752
 not in S (u,v) =  877 1754
 not in S (u,v) =  878 1756
 not in S (u,v) =  879 1758
 not in S (u,v) =  880 1760
 not in S (u,v) =  881 1762
 not in S (u,v) =  882 1764
 not in S (u,v) =  883 1766
 not in S (u,v) =  884 1768
 not in S (u,v) =  885 1770
u,v =  886 1772
 not in S (u,v) =  887 1774
 not in S (u,v) =  888 1776
 not in S (u,v) =  889 1778
 not in S (u,v) =  890 1780
 not in S (u,v) =  891 1782
 not in S (u,v) =  892 1784
 not in S (u,v) =  893 1786
 not in S (u,v) =  894 1788
u,v =  895 1790
 not in S (u,v) =  896 1792
 not in S (u,v) =  897 1794
 not in S (u,v) =  898 1796
 not in S (u,v) =  899 1798
 not in S (u,v) =  900 1800
 not in S (u,v) =  901 1802
u,v =  902 1804
u,v =  903 1806
u,v =  904 1808
 not in S (u,v) =  905 1810
u,v =  906 1812
u,v =  907 1814
 not in S (u,v) =  908 1816
 not in S (u,v) =  909 1818
 not in S (u,v) =  910 1820
 not in S (u,v) =  911 1822
 not in S (u,v) =  912 1824
 not in S (u,v) =  9

Graph created in time =  1.8841619491577148
random paths chosen in time =  0.949573278427124
projecting graph
isorted = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 323, 325, 327, 329, 331, 333, 335, 337, 339, 341, 343, 345, 347, 349, 351, 353, 355, 357, 359, 361, 363, 365, 367, 369, 371, 373,

 not in S (u,v) =  2 1186
 not in S (u,v) =  2 1198
 not in S (u,v) =  2 1202
 not in S (u,v) =  2 1214
 not in S (u,v) =  2 1226
 not in S (u,v) =  2 1234
 not in S (u,v) =  2 1238
 not in S (u,v) =  2 1262
 not in S (u,v) =  2 1282
 not in S (u,v) =  2 1286
 not in S (u,v) =  2 1294
 not in S (u,v) =  2 1306
 not in S (u,v) =  2 1318
 not in S (u,v) =  2 1322
 not in S (u,v) =  2 1346
 not in S (u,v) =  2 1354
 not in S (u,v) =  2 1366
 not in S (u,v) =  2 1382
 not in S (u,v) =  2 1402
 not in S (u,v) =  2 1418
 not in S (u,v) =  2 1438
 not in S (u,v) =  2 1454
 not in S (u,v) =  2 1466
 not in S (u,v) =  2 1478
 not in S (u,v) =  2 1486
 not in S (u,v) =  2 1502
 not in S (u,v) =  2 1514
 not in S (u,v) =  2 1522
 not in S (u,v) =  2 1538
 not in S (u,v) =  2 1546
 not in S (u,v) =  2 1574
 not in S (u,v) =  2 1594
 not in S (u,v) =  2 1618
 not in S (u,v) =  2 1622
 not in S (u,v) =  2 1642
 not in S (u,v) =  2 1646
 not in S (u,v) =  2 1654
 not in S (u,v) =  2 1658
 not in S (u

...

 not in S (u,v) =  3880 7760
 not in S (u,v) =  3881 7762
 not in S (u,v) =  3882 7764
u,v =  3883 7766
 not in S (u,v) =  3884 7768
 not in S (u,v) =  3885 7770
 not in S (u,v) =  3886 7772
 not in S (u,v) =  3887 7774
 not in S (u,v) =  3888 7776
 not in S (u,v) =  3889 7778
 not in S (u,v) =  3890 7780
 not in S (u,v) =  3891 7782
 not in S (u,v) =  3892 7784
u,v =  3893 7786
 not in S (u,v) =  3894 7788
 not in S (u,v) =  3895 7790
 not in S (u,v) =  3896 7792
 not in S (u,v) =  3897 7794
 not in S (u,v) =  3898 7796
u,v =  3899 7798
 not in S (u,v) =  3900 7800
 not in S (u,v) =  3901 7802
 not in S (u,v) =  3902 7804
 not in S (u,v) =  3903 7806
 not in S (u,v) =  3904 7808
 not in S (u,v) =  3905 7810
 not in S (u,v) =  3906 7812
 not in S (u,v) =  3907 7814
 not in S (u,v) =  3908 7816
 not in S (u,v) =  3909 7818
 not in S (u,v) =  3910 7820
 not in S (u,v) =  3911 7822
u,v =  3912 7824
 not in S (u,v) =  3913 7826
 not in S (u,v) =  3914 7828
 not in S (u,v) =  3915 7830
 not

 not in S (u,v) =  4828 9656
u,v =  4829 9658
 not in S (u,v) =  4830 9660
 not in S (u,v) =  4831 9662
 not in S (u,v) =  4832 9664
 not in S (u,v) =  4833 9666
 not in S (u,v) =  4834 9668
 not in S (u,v) =  4835 9670
 not in S (u,v) =  4836 9672
u,v =  4837 9674
 not in S (u,v) =  4838 9676
 not in S (u,v) =  4839 9678
 not in S (u,v) =  4840 9680
 not in S (u,v) =  4841 9682
 not in S (u,v) =  4842 9684
 not in S (u,v) =  4843 9686
 not in S (u,v) =  4844 9688
 not in S (u,v) =  4845 9690
 not in S (u,v) =  4846 9692
 not in S (u,v) =  4847 9694
 not in S (u,v) =  4848 9696
u,v =  4849 9698
 not in S (u,v) =  4850 9700
 not in S (u,v) =  4851 9702
u,v =  4852 9704
 not in S (u,v) =  4853 9706
 not in S (u,v) =  4854 9708
 not in S (u,v) =  4855 9710
 not in S (u,v) =  4856 9712
 not in S (u,v) =  4857 9714
 not in S (u,v) =  4858 9716
 not in S (u,v) =  4859 9718
 not in S (u,v) =  4860 9720
 not in S (u,v) =  4861 9722
 not in S (u,v) =  4862 9724
 not in S (u,v) =  4863 9726
 not

reweighting: (u,v) =  (4900, 9800) 0.05000000000000002 0.07692307692307694
reweighting: (u,v) =  (4800, 9600) 0.050000000000000044 0.07692307692307698
reweighting: (u,v) =  (2916, 5832) 0.09999999999999998 0.1538461538461538
reweighting: (u,v) =  (2916, 8748) 0.09999999999999998 0.1538461538461538
reweighting: (u,v) =  (4374, 8748) 0.09999999999999998 0.1538461538461538
graph projected in time =  6.348896503448486
paths =  [[236, 9676], [7337], [5995], [1147, 5735], [2916, 5832], [2762, 8286], [9413], [4548, 9096], [8575], [1290, 3870, 7740], [3776, 7552], [5788], [339, 678, 3390, 6780], [2668, 5336], [7625], [6826], [2192, 4384, 8768], [9337], [758, 3790, 7580], [4368, 8736], [5328], [6277], [8788], [900, 1800, 5400], [8511], [4918, 9836], [5140], [9474], [8150], [5926], [4819, 9638], [9772], [7189], [5243], [5834], [611, 1222, 3666, 7332], [1936, 3872, 7744], [4150, 8300], [8298], [7609], [1985, 3970, 7940], [7271], [3393, 6786], [3492, 6984], [7116], [6982], [5256], [2612, 7836], [4

shortest path as required: [4391, 8782] 0.0 0.0
shortest path as required: [754, 5278] 0.07692307692307693 0.07692307692307693
shortest path as required: [1198, 5990] 0.07692307692307693 0.07692307692307693
shortest path as required: [6363] 0 0
shortest path as required: [9980] 0 0
shortest path as required: [9610] 0 0
shortest path as required: [7536] 0 0
shortest path as required: [3175, 9525] 0.07692307692307693 0.07692307692307693
shortest path as required: [6153] 0 0
shortest path as required: [1474, 4422, 8844] 0.15384615384615385 0.15384615384615385
shortest path as required: [6161] 0 0
shortest path as required: [8924] 0 0
shortest path as required: [3175, 6350] 0.07692307692307693 0.07692307692307693
shortest path as required: [2566, 5132] 0.07692307692307693 0.07692307692307693
shortest path as required: [5106] 0 0
shortest path as required: [9552] 0 0
shortest path as required: [6379] 0 0
shortest path as required: [1318, 3954, 7908] 0.15384615384615385 0.15384615384615385
s

shortest path but not required: [3014, 9042] 1.0 1.0
shortest path but not required: [5983] 0 0
shortest path but not required: [4787, 9574] 1.0 1.0
shortest path but not required: [9155] 0 0
shortest path but not required: [5485] 0 0
shortest path as required: [1433, 7165] 0.0 0.0
shortest path but not required: [4365, 8730] 1.0 1.0
shortest path but not required: [9821] 0 0
shortest path but not required: [1820, 5460] 1.0 1.0
shortest path but not required: [6454] 0 0
shortest path but not required: [1878, 3756, 7512] 2.0 2.0
shortest path but not required: [4128, 8256] 1.0 1.0
shortest path but not required: [1544, 3088, 6176] 2.0 2.0
shortest path but not required: [9349] 0 0
shortest path but not required: [7145] 0 0
shortest path but not required: [8647] 0 0
shortest path but not required: [5067] 0 0
shortest path but not required: [5163] 0 0
shortest path but not required: [4605, 9210] 1.0 1.0
shortest path but not required: [8036] 0 0
shortest path but not required: [6304] 0 0


In [ ]:
from copy import deepcopy



# hier werden die isotonische Kosten gespeichert
isoG = deepcopy(myG)

np.random.seed(123)


s = -1 # neuer Knoten der eingefuegt wird
nn = list(myH.nodes())

# hier werden die momentan verwendeteten Kosten gespeichert.
# Annahme: C(u,v) = randuniform(0.01,10)*chat(u,v)
myH = deepcopy(myG) 

for e in myH.edges():
    u,v = e
    nx.set_edge_attributes(myH,{(u,v):{"weight":np.random.uniform(0.001,100)*myG.get_edge_data(u,v)["weight"]}})


tops = list(nx.topological_sort(myH))

for v in nn:
    if myH.in_degree(v)==0:
        print(v)
        myH.add_node(s)
        myH.add_edge(s,v,weight=0)
        print(myH.get_edge_data(s,v))

       
        
hh = nx.single_source_bellman_ford_path_length(myH,s,weight="weight")

myH.remove_node(s)

hl = []
for v in tops:
    hl.append(hh[v])
print(hl)        

In [ ]:
from sklearn.datasets import make_regression
from sklearn.isotonic import IsotonicRegression



X = np.array([[i] for i in range(1,len(hl)+1)])
y = np.array(hl)
#X, y = make_regression(n_samples=10, n_features=1, random_state=41)
print(X)
print(y)
iso_reg = IsotonicRegression(increasing=True).fit(X, y)
hisotonic = [h for h in iso_reg.predict(X)]

print(hisotonic)

hiso = dict(zip(tops,hisotonic))

isotoniccost = {}
for e in myH.edges():
    u,v = e
    isotoniccost[(u,v)] = newcost[(u,v)]+hiso[v]-hiso[u]
    nx.set_edge_attributes(isoG,{(u,v):{"weight":isotoniccost[(u,v)]}})
    print("(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u = ",(u,v),newcost[(u,v)],isotoniccost[(u,v)],hiso[v]-hiso[u])

(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (89, 9167) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (89, 9523) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (89, 9701) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (97, 194) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (97, 291) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (97, 485) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (97, 679) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (97, 1067) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (97, 1261) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (97, 16

(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (599, 6589) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (599, 7787) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (601, 1202) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (601, 1803) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (601, 3005) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (601, 4207) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (601, 6611) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (601, 7813) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (607, 1214) 1.0 28.231740120773907 27.231740120773907
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h

(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2689, 5378) 1.0 29.648908649313512 28.648908649313512
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2689, 8067) 1.0 29.648908649313512 28.648908649313512
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2693, 5386) 1.0 29.648908649313512 28.648908649313512
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2693, 8079) 1.0 29.648908649313512 28.648908649313512
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2699, 5398) 0.0 28.648908649313512 28.648908649313512
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2699, 8097) 1.0 29.648908649313512 28.648908649313512
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2707, 5414) 1.0 29.648908649313512 28.648908649313512
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2707, 8121) 1.0 29.648908649313512 28.648908649313512
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2711, 5422) 1.0 29.648908649313512 28.648908649313512
(u,v),newcost(u,v), isotoniccost[(u,v

(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (10, 370) 1.0 22.201192957534957 21.201192957534957
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (10, 410) 1.0 22.201192957534957 21.201192957534957
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (10, 430) 1.0 22.201192957534957 21.201192957534957
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (10, 470) 1.0 22.201192957534957 21.201192957534957
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (10, 530) 1.0 22.201192957534957 21.201192957534957
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (10, 590) 1.0 22.201192957534957 21.201192957534957
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (10, 610) 1.0 22.201192957534957 21.201192957534957
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (10, 670) 1.0 22.201192957534957 21.201192957534957
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (10, 710) 1.0 22.201192957534957 21.201192957534957
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (10, 730) 1.

(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (142, 6106) 1.0 18.109565810240838 17.109565810240838
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (142, 6674) 1.0 18.109565810240838 17.109565810240838
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (142, 7526) 1.0 18.109565810240838 17.109565810240838
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (142, 8378) 1.0 18.109565810240838 17.109565810240838
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (142, 8662) 1.0 18.109565810240838 17.109565810240838
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (142, 9514) 1.0 18.109565810240838 17.109565810240838
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (146, 292) 1.0 18.109565810240838 17.109565810240838
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (146, 438) 1.0 18.109565810240838 17.109565810240838
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (146, 730) 1.0 18.109565810240838 17.109565810240838
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u 

...

(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (532, 1064) 1.0 26.26199981655155 25.26199981655155
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (532, 1596) 1.0 26.26199981655155 25.26199981655155
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (532, 2660) 1.0 26.26199981655155 25.26199981655155
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (532, 3724) 1.0 26.26199981655155 25.26199981655155
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (532, 5852) 1.0 26.26199981655155 25.26199981655155
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (532, 6916) 1.0 26.26199981655155 25.26199981655155
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (532, 9044) 1.0 26.26199981655155 25.26199981655155
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (644, 1288) 1.0 26.26199981655155 25.26199981655155
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (644, 1932) 1.0 26.26199981655155 25.26199981655155
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (644, 3220) 

(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (180, 540) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (180, 900) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (180, 1260) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (180, 1980) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (180, 2340) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (180, 3060) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (180, 3420) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (180, 4140) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (180, 5220) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (180, 5580) 1.

(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (4944, 9888) 1.0 63.63574546280279 62.63574546280279
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (343, 686) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (343, 1029) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (343, 1715) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (343, 2401) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (343, 3773) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (343, 4459) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (343, 5831) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (343, 6517) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u

(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (3096, 9288) 1.0 47.70170785900905 46.70170785900905
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (3384, 6768) 1.0 47.70170785900905 46.70170785900905
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (3816, 7632) 0.0769230769230769 46.778630935932114 46.70170785900905
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (4248, 8496) 1.0 47.70170785900905 46.70170785900905
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (4392, 8784) 1.0 47.70170785900905 46.70170785900905
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (4824, 9648) 0.0769230769230769 58.12956263540957 58.052639558486504
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2738, 5476) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (2738, 8214) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (3034, 6068) 1.0 20.842442464743485 19.842442464743485
(u,v),newcost(u,v

(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (675, 2025) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (675, 3375) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (675, 4725) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (675, 7425) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (675, 8775) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (945, 1890) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (945, 2835) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (945, 4725) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (945, 6615) 1.0 41.74460863628943 40.74460863628943
(u,v),newcost(u,v), isotoniccost[(u,v)], h_v-h_u =  (1485, 2970)

In [ ]:
checkShortestPathProperty(isoG,S,isotoniccost,randomShortestPaths,paths,verbose=verbose)

In [ ]:
lossIso = 0
lossPrevious = 0

for e in myH.edges():
    u,v = e
    costH = myH.get_edge_data(u,v)["weight"]
    isoCost = isoG.get_edge_data(u,v)["weight"]
    previousCost = myG.get_edge_data(u,v)["weight"]
    lossIso += (costH-isoCost)**2
    lossPrevious += (costH-previousCost)**2

sqrtLossIso = np.sqrt(lossIso)
sqrtLossPrevious = np.sqrt(lossPrevious)

print("sqrtLossIso = ",sqrtLossIso)
print("sqrtLossPrevious = ",sqrtLossPrevious)
    

In [ ]:
import pandas as pd

def omega(n):
    return len(prime_divisors(n))


rows = []
for k in range(len(datapoints)):
    row = []
    row.append(datapoints[k][0])
    nrNodes = int(datapoints[k][0])
    nrEdges = sum(omega(k) for k in range(1,nrNodes+1))
    row.append(nrEdges)
    row.append(datapoints[k][2])
    row.append(graph_creation_times[k//3])
    row.append(random_path_times[k])
    row.append(graph_projection_times[k])
    print(row)
    rows.append(row)
#print(rows)    
result = pd.DataFrame(columns=["nrNodesInGraph","nrEdgesInGraph","nrRndPaths","graphCreationTimeInSec","rndPathCreationTimesInSec","graphProjectionTimesInSec"],data=rows)    

print(result)

result.to_csv("results-topological-sorting-kernel.csv")
